# Predicción de Series Temporales con RNN (LSTM) y Prophet

### Paso 1: Selección y Carga del Dataset

In [ ]:

import pandas as pd
import numpy as np

# Cargar el dataset
file_path = "air_quality.csv"  # Cambia esta ruta si es necesario
df = pd.read_csv(file_path)

# Inspeccionar los datos
print("Primeras filas del dataset:")
print(df.head())

# Información del dataset
print("\nInformación del dataset:")
print(df.info())

# Convertir la columna de tiempo a formato datetime si es necesario
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# Visualizar las primeras filas
df.head()


### Paso 2: Exploración y Preprocesamiento de Datos

In [ ]:

import matplotlib.pyplot as plt

# Visualizar la serie temporal
plt.figure(figsize=(12, 6))
plt.plot(df['value'], label='Serie Temporal Original')
plt.title('Serie Temporal')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.legend()
plt.show()

# Normalizar los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
df['scaled_value'] = scaler.fit_transform(df[['value']])

# Crear conjuntos de entrenamiento y prueba
train_size = int(len(df) * 0.8)
train, test = df[:train_size], df[train_size:]


### Paso 3: Implementación de la RNN con LSTM

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Crear generadores de series temporales
look_back = 10  # Número de pasos hacia atrás
train_generator = TimeseriesGenerator(train['scaled_value'], train['scaled_value'], length=look_back, batch_size=32)
test_generator = TimeseriesGenerator(test['scaled_value'], test['scaled_value'], length=look_back, batch_size=32)

# Crear el modelo LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(look_back, 1)),
    Dense(1)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
history = model.fit(train_generator, validation_data=test_generator, epochs=50, verbose=1)

# Visualizar las curvas de aprendizaje
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de Validación')
plt.title('Curvas de Pérdida durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (MSE)')
plt.legend()
plt.show()


### Paso 4: Predicción con Prophet

In [ ]:

from prophet import Prophet

# Preparar los datos para Prophet
prophet_df = df.reset_index()[['datetime', 'value']].rename(columns={'datetime': 'ds', 'value': 'y'})

# Crear y entrenar el modelo Prophet
prophet_model = Prophet()
prophet_model.fit(prophet_df[:train_size])

# Realizar predicciones
future = prophet_model.make_future_dataframe(periods=len(test), freq='H')
forecast = prophet_model.predict(future)

# Visualizar las predicciones
prophet_model.plot(forecast)
plt.title('Predicciones con Prophet')
plt.show()


### Paso 5: Comparación de Resultados

In [ ]:

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Predicciones con LSTM
lstm_predictions = model.predict(test_generator)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

# Calcular métricas de LSTM
lstm_rmse = np.sqrt(mean_squared_error(test['value'][look_back:], lstm_predictions))
lstm_mae = mean_absolute_error(test['value'][look_back:], lstm_predictions)

# Calcular métricas de Prophet
prophet_predictions = forecast.iloc[-len(test):]['yhat'].values
prophet_rmse = np.sqrt(mean_squared_error(test['value'], prophet_predictions))
prophet_mae = mean_absolute_error(test['value'], prophet_predictions)

print(f"LSTM RMSE: {lstm_rmse:.4f}, MAE: {lstm_mae:.4f}")
print(f"Prophet RMSE: {prophet_rmse:.4f}, MAE: {prophet_mae:.4f}")

# Visualizar las predicciones
plt.figure(figsize=(12, 6))
plt.plot(test['value'].values, label='Valores Reales', color='blue')
plt.plot(range(look_back, len(lstm_predictions) + look_back), lstm_predictions, label='Predicciones LSTM', color='red')
plt.plot(prophet_predictions, label='Predicciones Prophet', color='green')
plt.title('Comparación de Predicciones')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.legend()
plt.show()
